In [12]:
# seoul_lsit를 구별로 나누어 하기로 했습니다.
# 리스트에 도로명 주소를 지번주소로 바꿔 행정동을 구했었는데
# 안바뀌는 도로명이 있어서, 제가 맡은 구를 뽑아 직접 행정동을 구하려고 합니다.
import pandas as pd
df = pd.read_excel('seoul_list.xlsx')



In [13]:
def classify_gu(df,a):
  idx1 = df[df['시군구'] != a].index
  gu_df = df.drop(idx1)
  return gu_df 

In [11]:
a = '동작구 '
b = '마포구 '
c = '서대문구 '
d = '서초구 '
e = '동대문구 '
aa = classify_gu(df,a)
aa.to_excel('dongjack_df.xlsx')
bb = classify_gu(df,b)
bb.to_excel('mapo_df.xlsx')
cc = classify_gu(df,c)
cc.to_excel('seodaemoon_df.xlsx')
dd = classify_gu(df,d)
dd.to_excel('seocho_df.xlsx')
ee = classify_gu(df,e)
ee.to_excel('dongdaemoon_df.xlsx')



In [ ]:
# 수작업 완료

In [17]:
def classify_gu(df,a):
  idx1 = df[df['시군구'] != a].index
  gu_df = df.drop(idx1)
  return gu_df 

In [18]:
def classify_fast(df):
  f_df = df[df['타입'] != 'AC완속']
  idxs = f_df.index
  s_df = df.drop(idxs)
  return f_df, s_df

In [19]:
# 이 함수에 기준은 구마다 다른 것 같아서 수정할 필요가 있습니다.
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]

  full = pd.concat([f1,f2,f3,f4,f5,f6])
  temporary_c = pd.concat([c1, c2])
  fullfc = pd.concat([full, temporary_c])
  idx = fullfc.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [20]:
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c = pd.concat([temporary_c,c1,c2])
  nonc1 = df.drop(c1.index)
  nonc2 = nonc1.drop(c2.index)
  return c,nonc2

In [21]:
def classify_all(df,a):
  dfgu = classify_gu(df,a)
  # fast slow classify
  fast, slow = classify_fast(dfgu)
  # time classify
  fast_full , fast_nonfull , temporary_fast_cons = classify_time(fast) 
  slow_full , slow_nonfull , temporary_slow_cons = classify_time(slow)
  # constraints classify
  ffc, ffnc = classify_constraints(fast_full,temporary_fast_cons)
  nfc, nfnc = classify_constraints(fast_nonfull,temporary_fast_cons)
  fsc, fsnc = classify_constraints(slow_full, temporary_slow_cons)
  nsc, nsnc = classify_constraints(slow_nonfull, temporary_slow_cons)
  return ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc


In [31]:
#수작업된 파일을 불러와 위에 있는 함수를 적용해보겠습니다.
df = pd.read_excel('e_dongdaemoon_df.xlsx')
a = '동대문구 '
df.head()

,Unnamed: 0,Unnamed: 0.1,운영기관,충전소,충전기 ID,타입,지역,시군구,주소,이용가능시간,제한,종류,비고,Unnamed: 13,Unnamed: 13.1
0,72,1396,환경부(한국자동차환경협회),동대문구청,1,DC차데모+AC3상+DC콤보,서울특별시,동대문구,서울특별시 동대문구 천호대로 145 동대문구청 옥외주차장,24시간 이용가능,NaN,급속(50kW),주차요금 별도,[1] 행정동 명칭,용신동
1,151,2566,환경부(한국자동차환경협회),서울약령시 공영주차장,11,DC콤보,서울특별시,동대문구,서울특별시 동대문구 약령중앙로 26,24시간 이용가능,NaN,급속(200kW동시),10분/300원 친환경차 50% 감면,[1] 행정동 명칭,제기동
2,152,2567,환경부(한국자동차환경협회),서울약령시 공영주차장,12,DC콤보,서울특별시,동대문구,서울특별시 동대문구 약령중앙로 26,24시간 이용가능,NaN,급속(200kW동시),10분/300원 친환경차 50% 감면,[1] 행정동 명칭,제기동
3,234,3885,환경부(한국자동차환경협회),이마트 이문점,1,DC차데모+AC3상,서울특별시,동대문구,"서울특별시 동대문구 이문로 136, 지하주차장 1층 D-04(이문동 292-10)",10:00~24:00,NaN,급속(50kW),NaN,[1] 행정동 명칭,이문1동
4,250,4580,환경부(한국자동차환경협회),청량리롯데백화점,1,DC차데모+AC3상,서울특별시,동대문구,"서울특별시 동대문구 왕산로 214, 주차장 옥상(전농동 591-53)",24시간 이용가능,NaN,급속(50kW),NaN,[1] 행정동 명칭,전농1동


In [27]:
# 위 함수는 동대문구를 기준으로 만든것임으로 따로 함수정없이 사용하겠습니다.
# classify에서 구를 구분하는 함수는 이미 처리해서 필요없을것같네용
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)

In [32]:
gu_df = pd.DataFrame()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts(sort = True)

In [34]:
#동대문구 완료!
gu_df.to_excel('f_dong_df.xlsx')
gu_df.head()

,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
청량리동,NaN,4,NaN,NaN,5.0,6.0,5.0,NaN
답십리2동,NaN,2,NaN,1.0,NaN,7.0,NaN,7.0
답십리1동,NaN,2,NaN,1.0,NaN,36.0,NaN,NaN
휘경2동,NaN,2,NaN,5.0,4.0,10.0,4.0,6.0
제기동,NaN,2,NaN,NaN,10.0,6.0,10.0,2.0


In [35]:
df = pd.read_excel('e_dongjack_df.xlsx')
a = '동작구 '


In [36]:
df['이용가능시간'].value_counts()
# 24시간(단, 22시루근무자운영)이라는 기준이 하나 더 축가된것을 알수 있네요
# classify_time함수를 수정해야겠습니다.

24시간 이용가능                        187
09:00~18:00                       38
24시간 이용가능,입주민만 사용가능 거주자외출입제한      35
24시간                              11
~                                  8
주중/주말 : 06시~23시                    6
06:00~24:00                        5
09시~18시                            2
비개방(주차장협소, 관용차전용)                  2
24시간 이용가능,입주민만 사용가능 거주자 외 출입제      2
평일 09:00~18:00                     1
24시간(단, 22시이후근무자운영                 1
Name: 이용가능시간, dtype: int64

In [37]:
df['비고'].value_counts()
# 비고에도 기준이 더 추가되어있음을 알 수 있습니다.

입주민만 사용가능 거주자외 출입제한          35
입주민 등 거주자 외 출입 제한             8
공동주택 입주민 전용                   3
최초1시간 무료, 1시간 초과시 50%감면       1
주차요금 별도                       1
주차비 10분 100원(친환경차 50% 감면)     1
Name: 비고, dtype: int64

In [46]:
# 함수 수정
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  f7 = df[df['이용가능시간'] == '24시간(단, 22시이후근무자운영' ]
   
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]
  
 
  full = pd.concat([f1,f2,f3,f4,f5,f6])
  temporary_c = pd.concat([c1, c2])
  fullfc = pd.concat([full, temporary_c])
  idx = fullfc.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [38]:
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c3 = df[df['비고'] == '입주민 등 거주자 외 출입 제한' ]
  c4 = df[df['비고'] == '공동주택 입주민 전용' ]
 
  c = pd.concat([temporary_c,c1,c2,c3,c4])
  nonc1 = df.drop(c1.index)
  nonc2 = nonc1.drop(c2.index)
  return c,nonc2

In [40]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)

In [41]:
gu_df = pd.DataFrame()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts(sort = True)

In [42]:
#동작구 완료!
gu_df.to_excel('f_dongjack_df.xlsx')
gu_df.head()

,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
대방동,NaN,4,NaN,4.0,7.0,3.0,7.0,11.0
흑석동,NaN,2,NaN,1.0,NaN,16.0,NaN,13.0
사당2동,NaN,2,NaN,NaN,2.0,1.0,2.0,NaN
사당3동,NaN,1,NaN,NaN,NaN,28.0,3.0,6.0
상도3동,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df = pd.read_excel('e_mapo_df.xlsx')
a = '마포구 '


In [44]:
df['이용가능시간'].value_counts()
# 입주민 전용 24시간,주중/주말 : 24시간 라는 기준이 하나 더 축가된것을 알수 있네요
# classify_time함수를 수정해야겠습니다.

24시간 이용가능                       202
09:00~18:00                     102
24시간 이용가능,입주민만 사용가능 거주자외출입제한     61
24시간                             26
주중/주말 : 24시간                     10
주중 08~20시, 주말 09~18시              7
관계자전용 09:00~18:00                 6
입주민 전용 24시간                       4
06:00~18:00                       2
~                                 2
평일 09시~18시 이용가능                   2
08:00~24:00                       1
09:00~20:00                       1
Name: 이용가능시간, dtype: int64

In [45]:
df['비고'].value_counts()
#거주자외 출입제한이라는 기준이 더 있네여
# 24시간 이용가능은, 이용가능 시간에 NaN으로 주어져 있고
# 이것은 이미 24시간으로 처리함

입주민만 사용가능 거주자외 출입제한                     43
입주민만 사용가능 거주자 외 출입제한                    18
입주민 등 거주자 외 출입 제한                        8
24시간 이용가능                                8
5분당150원 저공해차량 50% 할인                     6
최초5분 250원/ 5분당 250원 친환경차 50% 감면          4
1시간 무료후 10분 초과마다 500원 부과                 4
5분당 200원 저공해차량 50% 할인                    3
공동주택 입주민 전용                              3
500분당 100원 저공해차량 50%할인                   2
주차요금 별도                                  2
최초30분 1500원, 10분당 500원 주민센터이용 50% 할인     2
거주자외 출입제한                                2
1시간 무료,5분 200원                           1
주차비 10분 300원                             1
10분/500원                                 1
주차비 10분 250원(친환경차 50% 감면)                1
Name: 비고, dtype: int64

In [69]:
# 함수 수정
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  f7 = df[df['이용가능시간'] == '24시간(단, 22시이후근무자운영' ]
  f7 = df[df['이용가능시간'] == '주중/주말 : 24시간'] 
   
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]
  c3 = df[df['이용가능시간'] == '입주민 전용 24시간' ]
  
 
  full = pd.concat([f1,f2,f3,f4,f5,f6,f7])
  temporary_c = pd.concat([c1, c2, c3])
  fullfc = pd.concat([full, temporary_c])
  idx = fullfc.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [70]:
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c3 = df[df['비고'] == '입주민 등 거주자 외 출입 제한' ]
  c4 = df[df['비고'] == '공동주택 입주민 전용' ]
  c5 = df[df['비고'] == '거주자외 출입제한' ]
 

  c = pd.concat([temporary_c,c1,c2,c3,c4,c5])
  nonc1 = df.drop(c1.index)
  nonc2 = nonc1.drop(c2.index)
  return c,nonc2

In [71]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)

In [72]:
gu_df = pd.DataFrame()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts(sort = True)

In [73]:
#마포구 완료!
gu_df.to_excel('f_mapo_df.xlsx')
gu_df.head()

,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
홍제1동,NaN,6,NaN,NaN,5.0,32,NaN,NaN
북가좌2동,NaN,4,NaN,NaN,NaN,14,NaN,1.0
남가좌2동,NaN,1,NaN,1.0,4.0,12,7.0,11.0
홍은1동,NaN,1,NaN,NaN,5.0,5,5.0,1.0
천연동,NaN,1,NaN,NaN,3.0,9,3.0,2.0


In [74]:
# 서초구
df = pd.read_excel('e_seocho_df.xlsx')
a = '서초구 '


In [75]:
df['이용가능시간'].value_counts()
# 24시간 이용가능(외부인 미개방) 이런 기준이 더 생겼네요

24시간 이용가능                           440
24시간 이용가능,입주민만 사용가능 거주자외출입제한        113
09:00~18:00                          97
24시간                                 46
12:00~13:00                          13
주중 08:00~19:00, 주말 09:00~18:00       12
주중/주말 : 06시~23시                      12
평일 09:00 ~ 18:00                     11
평일 09:00~18:00                        4
10:00~23:00                           4
06:30:00~22:30:59(휴일 불가)              3
24시간 이용가능(외부인 미개방)                    3
평일 09시~18시 이용가능, 소유주제한에따름             3
주중/주말 : 24시간                          2
평일 08:30~18:00 / 토요일 08:30~15:30      1
23시간 이용(09:00~08:00)                  1
09:00~24:00                           1
Name: 이용가능시간, dtype: int64

In [76]:
df['비고'].value_counts()
# 입주민으로 사용제한 이라는 기준이 추가됐네용


입주민만 사용가능 거주자외 출입제한       60
입주민만 사용가능 거주자 외 출입제한      53
입주민 등 거주자 외 출입 제한         12
10분당 300원 저공해차량 50% 할인     5
공동주택 입주민전용                 4
입주민으로 사용제한                 3
주차요금 별도                    2
24시간 이용가능                  2
기아차고객 주차비무료                1
10분/500원 친환경차 50% 감면       1
주차비 1시간 무료                 1
1시간 무료후 전기차 이용시 50% 할인     1
없음                         1
Name: 비고, dtype: int64

In [77]:
# 함수 수정
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  f7 = df[df['이용가능시간'] == '주중/주말 : 24시간']
  f8 = df[df['이용가능시간'] == '24시간(단, 22시이후근무자운영' ]
   
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]
  c3 = df[df['이용가능시간'] == '입주민 전용 24시간' ]
  c4 = df[df['이용가능시간'] == '24시간 이용가능(외부인 미개방)' ]
 
  full = pd.concat([f1,f2,f3,f4,f5,f6,f7,f8])
  temporary_c = pd.concat([c1, c2, c3,c4])
  fullfc = pd.concat([full, temporary_c])
  idx = fullfc.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [78]:
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c3 = df[df['비고'] == '입주민 등 거주자 외 출입 제한' ]
  c4 = df[df['비고'] == '공동주택 입주민 전용' ]
  c5 = df[df['비고'] == '거주자외 출입제한' ]
  c6 = df[df['비고'] == '입주민으로 사용제한' ]
  

  c = pd.concat([temporary_c,c1,c2,c3,c4,c5,c6])
  nonc1 = df.drop(c1.index)
  nonc2 = nonc1.drop(c2.index)
  return c,nonc2

In [85]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)

In [80]:
gu_df = pd.DataFrame()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts(sort = True)

In [81]:
# 서초구 완료!
gu_df.to_excel('f_seocho_df.xlsx')
gu_df.head()

,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
서초4동,NaN,13,NaN,NaN,22.0,28,22.0,11
양재2동,NaN,10,NaN,17.0,NaN,22,NaN,6
서초3동,NaN,7,NaN,1.0,NaN,73,NaN,19
반포1동,NaN,6,NaN,1.0,57.0,69,54.0,7
내곡동,NaN,5,NaN,2.0,6.0,24,6.0,5


In [62]:
# 마지막 서대문구~!
df = pd.read_excel('e_seodaemoon_df.xlsx')
a = '서대문구 '



In [63]:
df['이용가능시간'].value_counts()
#새로운 기준이 없네요 ㅎㅎ

24시간 이용가능                       163
24시간 이용가능,입주민만 사용가능 거주자외출입제한     82
09:00~18:00                      16
24시간                             10
평일 09:00~18:00                    8
06:00-24:00                       7
관계자전용 09:00~18:00                 1
08:00~24:00                       1
비개방, 관용차 전용, 주차장협소                1
주중/주말 : 24시간                      1
Name: 이용가능시간, dtype: int64

In [82]:
df['비고'].value_counts()
#이것도 새로운 기준이 없으니 함수 수정없이 바로 할 수 있겠다~ 무야호~!

입주민만 사용가능 거주자외 출입제한       60
입주민만 사용가능 거주자 외 출입제한      53
입주민 등 거주자 외 출입 제한         12
10분당 300원 저공해차량 50% 할인     5
공동주택 입주민전용                 4
입주민으로 사용제한                 3
주차요금 별도                    2
24시간 이용가능                  2
기아차고객 주차비무료                1
10분/500원 친환경차 50% 감면       1
주차비 1시간 무료                 1
1시간 무료후 전기차 이용시 50% 할인     1
없음                         1
Name: 비고, dtype: int64

In [86]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)

In [87]:
gu_df = pd.DataFrame()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts(sort = True)
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts(sort = True)

In [88]:
# 서대문구 완료!
gu_df.to_excel('f_seodaemoon_df.xlsx')
gu_df.head()

,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
서초4동,NaN,13,NaN,NaN,22.0,28,22.0,11
양재2동,NaN,10,NaN,17.0,NaN,22,NaN,6
서초3동,NaN,7,NaN,1.0,NaN,73,NaN,19
반포1동,NaN,6,NaN,1.0,57.0,69,54.0,7
내곡동,NaN,5,NaN,2.0,6.0,24,6.0,5


In [ ]:
# 여태 만들었던 DataFrame을 하나로 합쳐봅시다!

In [90]:
a = pd.read_excel('f_dong_df.xlsx')
b = pd.read_excel('f_dongjack_df.xlsx')
c = pd.read_excel('f_mapo_df.xlsx')
d = pd.read_excel('f_seocho_df.xlsx')
e = pd.read_excel('f_seodaemoon_df.xlsx')

In [92]:
final_df = pd.concat([a,b,c,d,e])

In [94]:
final_df.to_excel('5gu_df.xlsx')
final_df.head()

,Unnamed: 0,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
0,청량리동,NaN,4,NaN,NaN,5.0,6.0,5.0,NaN
1,답십리2동,NaN,2,NaN,1.0,NaN,7.0,NaN,7.0
2,답십리1동,NaN,2,NaN,1.0,NaN,36.0,NaN,NaN
3,휘경2동,NaN,2,NaN,5.0,4.0,10.0,4.0,6.0
4,제기동,NaN,2,NaN,NaN,10.0,6.0,10.0,2.0
